In [1]:
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import trimesh
import argparse
import os
import time
import numpy as np
import torch
import torch.optim as optim
import lib.utils as utils
from lib.utils import standard_normal_logprob
from lib.utils import count_nfe, count_total_time
from lib.utils import build_model_tabular

from lib.visualize_flow import visualize_transform
import lib.layers.odefunc as odefunc
from matplotlib import pyplot as plt
%matplotlib inline 

In [2]:
SOLVERS = ["dopri5"]
parser = argparse.ArgumentParser('SoftFlow')
parser.add_argument(
    '--data', choices=['2spirals_1d','2spirals_2d', 'swissroll_1d','swissroll_2d', 'circles_1d', 'circles_2d', '2sines_1d', 'target_1d'],
    type=str, default='2spirals_1d'
)
parser.add_argument("--layer_type", type=str, default="concatsquash", choices=["concatsquash"])
parser.add_argument('--dims', type=str, default='64-64-64')
parser.add_argument("--num_blocks", type=int, default=1, help='Number of stacked CNFs.')
parser.add_argument('--time_length', type=float, default=0.5)
parser.add_argument('--train_T', type=eval, default=True)
parser.add_argument("--divergence_fn", type=str, default="brute_force", choices=["brute_force", "approximate"])
parser.add_argument("--nonlinearity", type=str, default="tanh", choices=odefunc.NONLINEARITIES)

parser.add_argument('--solver', type=str, default='dopri5', choices=SOLVERS)
parser.add_argument('--atol', type=float, default=1e-5)
parser.add_argument('--rtol', type=float, default=1e-5)

parser.add_argument('--residual', type=eval, default=False, choices=[True, False])
parser.add_argument('--rademacher', type=eval, default=False, choices=[True, False])
parser.add_argument('--spectral_norm', type=eval, default=False, choices=[True, False])
parser.add_argument('--niters', type=int, default=36000)
parser.add_argument('--batch_size', type=int, default=100)
parser.add_argument('--test_batch_size', type=int, default=1000)
parser.add_argument('--lr', type=float, default=1e-3)
parser.add_argument('--weight_decay', type=float, default=1e-5)

# for the proposed method
parser.add_argument('--std_min', type=float, default=0.0)
parser.add_argument('--std_max', type=float, default=0.1)
parser.add_argument('--std_weight', type=float, default=2)

parser.add_argument('--viz_freq', type=int, default=100)
parser.add_argument('--val_freq', type=int, default=400)
parser.add_argument('--log_freq', type=int, default=10)
parser.add_argument('--gpu', type=int, default=0)
# args = parser.parse_args()
args, unknown = parser.parse_known_args()


In [3]:
# logger
save_path = './results/' + args.data + '/SoftFlow'
utils.makedirs(save_path)

device = torch.device('cuda:' + str(args.gpu) if torch.cuda.is_available() else 'cpu')

In [4]:
def get_transforms(model):

    def sample_fn(z, logpz=None):
        zeors_std = torch.zeros(z.shape[0], 1).to(z)
        if logpz is not None:
            return model(z, zeors_std, logpz, reverse=True)
        else:
            return model(z, zeors_std, reverse=True)

    def density_fn(x, logpx=None):
        zeors_std = torch.zeros(x.shape[0], 1).to(x)
        if logpx is not None:
            return model(x, zeors_std, logpx, reverse=False)
        else:
            return model(x, zeors_std, reverse=False)

    return sample_fn, density_fn

DATA = trimesh.load('1.ply')[:]
DATA -= np.mean(DATA, axis=0) 
DATA = (DATA-np.min(DATA, axis=0))/(np.max(DATA, axis=0)-np.min(DATA, axis=0))
DATA = DATA*3
def compute_loss(args, model, batch_size=None):
    if batch_size is None: batch_size = args.batch_size

    # load data
#     x = toy_data.inf_train_gen(args.data, batch_size=batch_size)
    x = DATA[:,1:3]


    x = torch.from_numpy(x).type(torch.float32).to(device)
    zero = torch.zeros(x.shape[0], 1).to(x)

    # transform to z
    std = (args.std_max - args.std_min) * torch.rand_like(x[:,0]).view(-1,1) + args.std_min
    eps = torch.randn_like(x) * std
    std_in = std / args.std_max * args.std_weight
    z, delta_logp = model(x+eps, std_in, zero)

    # compute log q(z)
    logpz = standard_normal_logprob(z).sum(1, keepdim=True)

    logpx = logpz - delta_logp
    loss = -torch.mean(logpx)
    return loss

In [5]:
model = build_model_tabular(args, 2).to(device)

optimizer = optim.Adam(model.parameters(), lr=args.lr, weight_decay=args.weight_decay)

time_meter = utils.RunningAverageMeter(0.93)
loss_meter = utils.RunningAverageMeter(0.93)
nfef_meter = utils.RunningAverageMeter(0.93)
nfeb_meter = utils.RunningAverageMeter(0.93)
tt_meter = utils.RunningAverageMeter(0.93)

end = time.time()
best_loss = float('inf')

In [ ]:
model.train()
for itr in range(1, args.niters + 1):
    optimizer.zero_grad()

    loss = compute_loss(args, model)

    loss_meter.update(loss.item())

    total_time = count_total_time(model)
    nfe_forward = count_nfe(model)

    loss.backward()
    optimizer.step()

    nfe_total = count_nfe(model)
    nfe_backward = nfe_total - nfe_forward
    nfef_meter.update(nfe_forward)
    nfeb_meter.update(nfe_backward)

    time_meter.update(time.time() - end)
    tt_meter.update(total_time)

    log_message = (
        'Iter {:04d} | Time {:.4f}({:.4f}) | Loss {:.6f}({:.6f}) | NFE Forward {:.0f}({:.1f})'
        ' | NFE Backward {:.0f}({:.1f}) | CNF Time {:.4f}({:.4f})'.format(
            itr, time_meter.val, time_meter.avg, loss_meter.val, loss_meter.avg, nfef_meter.val, nfef_meter.avg,
            nfeb_meter.val, nfeb_meter.avg, tt_meter.val, tt_meter.avg
        )
    )

    print(log_message)

    if itr % args.val_freq == 0 or itr == args.niters:
        with torch.no_grad():
            model.eval()
            test_loss = compute_loss(args, model, batch_size=args.test_batch_size)
            test_nfe = count_nfe(model)
            log_message = '[TEST] Iter {:04d} | Test Loss {:.6f} | NFE {:.0f}'.format(itr, test_loss, test_nfe)
            print(log_message)

            if test_loss.item() < best_loss:
                best_loss = test_loss.item()
                utils.makedirs(save_path)
                torch.save({
                    'args': args,
                    'state_dict': model.state_dict(),
                }, os.path.join(save_path, 'checkpt.pth'))
            model.train()

    if itr % args.viz_freq == 0:
        with torch.no_grad():
            model.eval()
#             p_samples = toy_data.inf_train_gen(args.data, batch_size=2000)
            p_samples = DATA[:,1:3]

            sample_fn, density_fn = get_transforms(model)

            plt.figure(figsize=(9, 3))
            visualize_transform(
                p_samples, torch.randn, standard_normal_logprob, transform=sample_fn, inverse_transform=density_fn,
                samples=True, npts=200, device=device
            )
            fig_filename = os.path.join(save_path, 'figs', '{:04d}.jpg'.format(itr))
            utils.makedirs(os.path.dirname(fig_filename))
            plt.savefig(fig_filename, format='png', dpi=1200)
            plt.close()
            model.train()

    end = time.time()

print('Training has finished.')

Iter 0001 | Time 7.2402(7.2402) | Loss 4.495609(4.495609) | NFE Forward 20(20.0) | NFE Backward 26(26.0) | CNF Time 0.5000(0.5000)
Iter 0002 | Time 0.2389(6.7501) | Loss 4.459622(4.493090) | NFE Forward 20(20.0) | NFE Backward 26(26.0) | CNF Time 0.5000(0.5000)
Iter 0003 | Time 0.1643(6.2891) | Loss 4.418822(4.487891) | NFE Forward 20(20.0) | NFE Backward 26(26.0) | CNF Time 0.5000(0.5000)
Iter 0004 | Time 0.1667(5.8605) | Loss 4.382850(4.480538) | NFE Forward 20(20.0) | NFE Backward 26(26.0) | CNF Time 0.5000(0.5000)
Iter 0005 | Time 0.1648(5.4618) | Loss 4.357794(4.471946) | NFE Forward 20(20.0) | NFE Backward 26(26.0) | CNF Time 0.5000(0.5000)
Iter 0006 | Time 0.1617(5.0908) | Loss 4.320683(4.461358) | NFE Forward 20(20.0) | NFE Backward 26(26.0) | CNF Time 0.5000(0.5000)
Iter 0007 | Time 0.1581(4.7455) | Loss 4.294074(4.449648) | NFE Forward 20(20.0) | NFE Backward 26(26.0) | CNF Time 0.5000(0.5000)
Iter 0008 | Time 0.1603(4.4246) | Loss 4.240463(4.435005) | NFE Forward 20(20.0) | 

Iter 0064 | Time 0.1834(0.2517) | Loss 2.352248(2.695282) | NFE Forward 26(25.2) | NFE Backward 26(26.0) | CNF Time 0.5000(0.5000)
Iter 0065 | Time 0.1797(0.2467) | Loss 2.327684(2.669550) | NFE Forward 26(25.2) | NFE Backward 26(26.0) | CNF Time 0.5000(0.5000)
Iter 0066 | Time 0.1818(0.2421) | Loss 2.318443(2.644973) | NFE Forward 26(25.3) | NFE Backward 26(26.0) | CNF Time 0.5000(0.5000)
Iter 0067 | Time 0.1794(0.2377) | Loss 2.305924(2.621239) | NFE Forward 26(25.3) | NFE Backward 26(26.0) | CNF Time 0.5000(0.5000)
Iter 0068 | Time 0.1814(0.2338) | Loss 2.289919(2.598047) | NFE Forward 26(25.4) | NFE Backward 26(26.0) | CNF Time 0.5000(0.5000)
Iter 0069 | Time 0.1778(0.2299) | Loss 2.279443(2.575745) | NFE Forward 26(25.4) | NFE Backward 26(26.0) | CNF Time 0.5000(0.5000)
Iter 0070 | Time 0.1828(0.2266) | Loss 2.276952(2.554829) | NFE Forward 26(25.5) | NFE Backward 26(26.0) | CNF Time 0.5000(0.5000)
Iter 0071 | Time 0.1795(0.2233) | Loss 2.261255(2.534279) | NFE Forward 26(25.5) | 

Iter 0127 | Time 0.1815(0.1763) | Loss 2.138875(2.148816) | NFE Forward 26(24.0) | NFE Backward 26(26.0) | CNF Time 0.5000(0.5000)
Iter 0128 | Time 0.1634(0.1754) | Loss 2.134129(2.147788) | NFE Forward 20(23.7) | NFE Backward 26(26.0) | CNF Time 0.5000(0.5000)
Iter 0129 | Time 0.1826(0.1759) | Loss 2.135770(2.146947) | NFE Forward 26(23.9) | NFE Backward 26(26.0) | CNF Time 0.5000(0.5000)
Iter 0130 | Time 0.1794(0.1762) | Loss 2.133312(2.145993) | NFE Forward 26(24.0) | NFE Backward 26(26.0) | CNF Time 0.5000(0.5000)
Iter 0131 | Time 0.1812(0.1765) | Loss 2.138623(2.145477) | NFE Forward 26(24.2) | NFE Backward 26(26.0) | CNF Time 0.5000(0.5000)
Iter 0132 | Time 0.1633(0.1756) | Loss 2.134988(2.144742) | NFE Forward 20(23.9) | NFE Backward 26(26.0) | CNF Time 0.5000(0.5000)
Iter 0133 | Time 0.1657(0.1749) | Loss 2.135451(2.144092) | NFE Forward 20(23.6) | NFE Backward 26(26.0) | CNF Time 0.5000(0.5000)
Iter 0134 | Time 0.1798(0.1752) | Loss 2.133564(2.143355) | NFE Forward 26(23.8) | 

Iter 0191 | Time 0.1664(0.1685) | Loss 2.117535(2.123160) | NFE Forward 20(21.0) | NFE Backward 26(26.0) | CNF Time 0.5000(0.5000)
Iter 0192 | Time 0.1630(0.1682) | Loss 2.118832(2.122857) | NFE Forward 20(20.9) | NFE Backward 26(26.0) | CNF Time 0.5000(0.5000)
Iter 0193 | Time 0.1702(0.1683) | Loss 2.114367(2.122263) | NFE Forward 20(20.9) | NFE Backward 26(26.0) | CNF Time 0.5000(0.5000)
Iter 0194 | Time 0.1657(0.1681) | Loss 2.118317(2.121987) | NFE Forward 20(20.8) | NFE Backward 26(26.0) | CNF Time 0.5000(0.5000)
Iter 0195 | Time 0.1678(0.1681) | Loss 2.113635(2.121402) | NFE Forward 20(20.7) | NFE Backward 26(26.0) | CNF Time 0.5000(0.5000)
Iter 0196 | Time 0.1656(0.1679) | Loss 2.114673(2.120931) | NFE Forward 20(20.7) | NFE Backward 26(26.0) | CNF Time 0.5000(0.5000)
Iter 0197 | Time 0.1674(0.1679) | Loss 2.110229(2.120182) | NFE Forward 20(20.6) | NFE Backward 26(26.0) | CNF Time 0.5000(0.5000)
Iter 0198 | Time 0.1800(0.1687) | Loss 2.112741(2.119661) | NFE Forward 26(21.0) | 

Iter 0255 | Time 0.1989(0.1829) | Loss 2.013272(2.052442) | NFE Forward 32(26.3) | NFE Backward 26(26.0) | CNF Time 0.5000(0.5000)
Iter 0256 | Time 0.1951(0.1837) | Loss 2.002604(2.048953) | NFE Forward 32(26.7) | NFE Backward 26(26.0) | CNF Time 0.5000(0.5000)
Iter 0257 | Time 0.1980(0.1847) | Loss 2.004903(2.045870) | NFE Forward 32(27.1) | NFE Backward 26(26.0) | CNF Time 0.5000(0.5000)
Iter 0258 | Time 0.1950(0.1854) | Loss 1.994777(2.042293) | NFE Forward 32(27.4) | NFE Backward 26(26.0) | CNF Time 0.5000(0.5000)
Iter 0259 | Time 0.1982(0.1863) | Loss 1.988469(2.038525) | NFE Forward 32(27.7) | NFE Backward 26(26.0) | CNF Time 0.5000(0.5000)
Iter 0260 | Time 0.1933(0.1868) | Loss 1.982203(2.034583) | NFE Forward 32(28.0) | NFE Backward 26(26.0) | CNF Time 0.5000(0.5000)
Iter 0261 | Time 0.1989(0.1877) | Loss 1.981406(2.030860) | NFE Forward 32(28.3) | NFE Backward 26(26.0) | CNF Time 0.5000(0.5000)
Iter 0262 | Time 0.1960(0.1882) | Loss 1.972298(2.026761) | NFE Forward 32(28.6) | 

...

Iter 2712 | Time 0.3187(0.3102) | Loss 1.431319(1.430187) | NFE Forward 50(50.0) | NFE Backward 44(41.8) | CNF Time 0.5000(0.5000)
Iter 2713 | Time 0.2961(0.3092) | Loss 1.430158(1.430185) | NFE Forward 50(50.0) | NFE Backward 38(41.6) | CNF Time 0.5000(0.5000)
Iter 2714 | Time 0.3207(0.3100) | Loss 1.429333(1.430126) | NFE Forward 50(50.0) | NFE Backward 44(41.7) | CNF Time 0.5000(0.5000)
Iter 2715 | Time 0.2949(0.3090) | Loss 1.434806(1.430453) | NFE Forward 50(50.0) | NFE Backward 38(41.5) | CNF Time 0.5000(0.5000)
Iter 2716 | Time 0.3203(0.3098) | Loss 1.430437(1.430452) | NFE Forward 50(50.0) | NFE Backward 44(41.6) | CNF Time 0.5000(0.5000)
Iter 2717 | Time 0.2949(0.3087) | Loss 1.425733(1.430122) | NFE Forward 50(50.0) | NFE Backward 38(41.4) | CNF Time 0.5000(0.5000)
Iter 2718 | Time 0.2953(0.3078) | Loss 1.428055(1.429977) | NFE Forward 50(50.0) | NFE Backward 38(41.1) | CNF Time 0.5000(0.5000)
Iter 2719 | Time 0.2954(0.3069) | Loss 1.429231(1.429925) | NFE Forward 50(50.0) | 

Iter 2775 | Time 0.3171(0.3147) | Loss 1.429848(1.427967) | NFE Forward 50(50.0) | NFE Backward 44(43.0) | CNF Time 0.5000(0.5000)
Iter 2776 | Time 0.3179(0.3149) | Loss 1.430613(1.428152) | NFE Forward 50(50.0) | NFE Backward 44(43.0) | CNF Time 0.5000(0.5000)
Iter 2777 | Time 0.3164(0.3150) | Loss 1.432193(1.428435) | NFE Forward 50(50.0) | NFE Backward 44(43.1) | CNF Time 0.5000(0.5000)
Iter 2778 | Time 0.3177(0.3152) | Loss 1.432125(1.428693) | NFE Forward 50(50.0) | NFE Backward 44(43.2) | CNF Time 0.5000(0.5000)
Iter 2779 | Time 0.3169(0.3153) | Loss 1.428652(1.428690) | NFE Forward 50(50.0) | NFE Backward 44(43.2) | CNF Time 0.5000(0.5000)
Iter 2780 | Time 0.3173(0.3155) | Loss 1.425342(1.428456) | NFE Forward 50(50.0) | NFE Backward 44(43.3) | CNF Time 0.5000(0.5000)
Iter 2781 | Time 0.3184(0.3157) | Loss 1.429025(1.428496) | NFE Forward 50(50.0) | NFE Backward 44(43.3) | CNF Time 0.5000(0.5000)
Iter 2782 | Time 0.3169(0.3158) | Loss 1.429097(1.428538) | NFE Forward 50(50.0) | 

Iter 2838 | Time 0.3173(0.3165) | Loss 1.436573(1.429775) | NFE Forward 50(50.0) | NFE Backward 44(43.3) | CNF Time 0.5000(0.5000)
Iter 2839 | Time 0.3183(0.3166) | Loss 1.426361(1.429536) | NFE Forward 50(50.0) | NFE Backward 44(43.3) | CNF Time 0.5000(0.5000)
Iter 2840 | Time 0.3190(0.3168) | Loss 1.425475(1.429251) | NFE Forward 50(50.0) | NFE Backward 44(43.4) | CNF Time 0.5000(0.5000)
Iter 2841 | Time 0.3183(0.3169) | Loss 1.425929(1.429019) | NFE Forward 50(50.0) | NFE Backward 44(43.4) | CNF Time 0.5000(0.5000)
Iter 2842 | Time 0.3179(0.3169) | Loss 1.423581(1.428638) | NFE Forward 50(50.0) | NFE Backward 44(43.4) | CNF Time 0.5000(0.5000)
Iter 2843 | Time 0.3225(0.3173) | Loss 1.429368(1.428689) | NFE Forward 50(50.0) | NFE Backward 44(43.5) | CNF Time 0.5000(0.5000)
Iter 2844 | Time 0.3188(0.3174) | Loss 1.429948(1.428777) | NFE Forward 50(50.0) | NFE Backward 44(43.5) | CNF Time 0.5000(0.5000)
Iter 2845 | Time 0.3181(0.3175) | Loss 1.429207(1.428807) | NFE Forward 50(50.0) | 

Iter 2901 | Time 0.3222(0.3232) | Loss 1.432268(1.427543) | NFE Forward 50(50.0) | NFE Backward 44(43.6) | CNF Time 0.5000(0.5000)
Iter 2902 | Time 0.3389(0.3243) | Loss 1.428828(1.427633) | NFE Forward 50(50.0) | NFE Backward 44(43.6) | CNF Time 0.5000(0.5000)
Iter 2903 | Time 0.3384(0.3253) | Loss 1.429646(1.427774) | NFE Forward 50(50.0) | NFE Backward 44(43.6) | CNF Time 0.5000(0.5000)
Iter 2904 | Time 0.3299(0.3256) | Loss 1.430056(1.427934) | NFE Forward 50(50.0) | NFE Backward 44(43.7) | CNF Time 0.5000(0.5000)
Iter 2905 | Time 0.3412(0.3267) | Loss 1.430523(1.428115) | NFE Forward 50(50.0) | NFE Backward 44(43.7) | CNF Time 0.5000(0.5000)
Iter 2906 | Time 0.3514(0.3284) | Loss 1.428143(1.428117) | NFE Forward 50(50.0) | NFE Backward 44(43.7) | CNF Time 0.5000(0.5000)
Iter 2907 | Time 0.3364(0.3290) | Loss 1.433322(1.428481) | NFE Forward 50(50.0) | NFE Backward 44(43.7) | CNF Time 0.5000(0.5000)
Iter 2908 | Time 0.3355(0.3295) | Loss 1.429830(1.428576) | NFE Forward 50(50.0) | 

Iter 2964 | Time 0.3530(0.3342) | Loss 1.426923(1.428397) | NFE Forward 50(50.0) | NFE Backward 44(44.0) | CNF Time 0.5000(0.5000)
Iter 2965 | Time 0.3521(0.3355) | Loss 1.422968(1.428017) | NFE Forward 50(50.0) | NFE Backward 44(44.0) | CNF Time 0.5000(0.5000)
Iter 2966 | Time 0.3315(0.3352) | Loss 1.422964(1.427663) | NFE Forward 50(50.0) | NFE Backward 44(44.0) | CNF Time 0.5000(0.5000)
Iter 2967 | Time 0.3424(0.3357) | Loss 1.433240(1.428054) | NFE Forward 50(50.0) | NFE Backward 44(44.0) | CNF Time 0.5000(0.5000)
Iter 2968 | Time 0.3324(0.3355) | Loss 1.428709(1.428099) | NFE Forward 50(50.0) | NFE Backward 44(44.0) | CNF Time 0.5000(0.5000)
Iter 2969 | Time 0.3748(0.3382) | Loss 1.430720(1.428283) | NFE Forward 50(50.0) | NFE Backward 44(44.0) | CNF Time 0.5000(0.5000)
Iter 2970 | Time 0.3614(0.3398) | Loss 1.426257(1.428141) | NFE Forward 50(50.0) | NFE Backward 44(44.0) | CNF Time 0.5000(0.5000)


In [7]:
plt.plot(loss[:, 1], loss[:,2], 'o')

IndexError: too many indices for tensor of dimension 0

In [ ]:
loss.shape